In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer.py


In [3]:
%%writefile combiner.py

import sys

current_key = None
word_sum = 0
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)  

Writing combiner.py


In [4]:
%%writefile reducer2.py

import sys

def update_top(top_list, pair):
    top_list.append(pair)
    sorted_by_value = sorted(top_list, key=lambda tup: tup[1], reverse=True)
    return sorted_by_value[:10]  

current_key = None
word_sum = 0
global_counter = 0

top_seven = []

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            top_seven = update_top(top_seven, (current_key, word_sum))
            global_counter += word_sum
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    top_seven = update_top(top_seven, (current_key, word_sum))
    global_counter += word_sum

#print global_counter
#print top_seven
print "%s\t%d" % (top_seven[6][0], top_seven[6][1])

'''for key, val in top_seven:
    print '%s\t%s' % (key, val)'''

Writing reducer2.py


In [5]:
%%bash
INTERM_OUT="interm"
OUT_DIR="wordcount_result"

hdfs dfs -rm -r -skipTrash ${INTERM_OUT} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Job1" \
    -D mapreduce.job.reduces=3 \
    -files mapper.py,reducer.py,combiner.py \
    -mapper "python mapper.py" \
    -combiner "python combiner.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${INTERM_OUT} > /dev/null
    
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Job2" \
    -D mapreduce.job.reduces=1 \
    -files reducer2.py \
    -mapper "cat" \
    -reducer "python reducer2.py" \
    -input ${INTERM_OUT}/part-0000* \
    -output ${OUT_DIR} > /dev/null    

hdfs dfs -cat ${OUT_DIR}/part-0000* | head        

is	126420


rm: `interm': No such file or directory
rm: `wordcount_result': No such file or directory
17/11/12 10:26:29 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/12 10:26:31 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/12 10:26:36 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/12 10:26:37 INFO mapreduce.JobSubmitter: number of splits:2
17/11/12 10:26:37 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/11/12 10:26:38 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1510482275980_0001
17/11/12 10:26:40 INFO impl.YarnClientImpl: Submitted application application_1510482275980_0001
17/11/12 10:26:40 INFO mapreduce.Job: The url to track the job: http://2ab83dfdbd1b:8088/proxy/application_1510482275980_0001/
17/11/12 10:26:40 INFO mapreduce.Job: Running job: job_1510482275980_0001
17/11/12 10:27:18 INFO mapreduce.Job: Job job_1510482275980_0001 running in uber mod